# Strain-Gage Low Noise Signal Conditioning

<img src="images/architecture.png" width="600">

<img src="images/Phase1Specs.png" width="600">

#### Step X: Use the first stage to realize the required gain. This will minimize the impact of U3’s noise, as well as that of R1and R2.

Target for 80dB of gain. <br>
Choose $U_1$ and $U_2$ to have 60dB gain and $U_3$ to have 20dB.

$\begin{eqnarray} 
&& \frac{V_{out}}{V_{in}} &=& 1+2\frac{R_F}{R_G} \\[0.5em]
\end{eqnarray}$

#### Step X: Select  component  values  that  allow  you  to  achieve  the  noise  target,  with  resistor tolerances that make it possible to meet the CMRR specification.

#### Step 1 : Determine the input-referred voltage noise density required to meet the specifications.

#### Step 2 : Perform noise analysis of the instrumentation amplifier and determine the relative impact of all noise sources on the input-referred noise.

#### Step 3 : Be sure to include the noise from the strain gage resistance.You shouldmodel the strain gage as a Thevenin equivalent circuit.